# Medicare Provider Utilization and Payment Data: 2014 Part D Prescriber

[Data Source](https://data.cms.gov/Medicare-Claims/Medicare-Provider-Utilization-and-Payment-Data-201/465c-49pb)

In [111]:
import urllib.request
url = 'https://data.cms.gov/resource/uggq-gnqc.json?$limit=1000'
data_bytes = urllib.request.urlopen(url).read()

In [112]:
import json
data_string = data_bytes.decode('utf-8')
data_json = json.loads(str(data_string))
data_json[0]

{'bene_count': '12',
 'bene_count_ge65': '12',
 'description_flag': 'S',
 'drug_name': 'POTASSIUM CHLORIDE',
 'generic_name': 'POTASSIUM CHLORIDE',
 'npi': '1821069519',
 'nppes_provider_city': 'ARDMORE',
 'nppes_provider_first_name': 'SUSAN',
 'nppes_provider_last_org_name': 'SANDLER',
 'nppes_provider_state': 'PA',
 'specialty_desc': 'Family Practice',
 'total_30_day_fill_count': '66',
 'total_30_day_fill_count_ge65': '66',
 'total_claim_count': '46',
 'total_claim_count_ge65': '46',
 'total_day_supply': '1939',
 'total_day_supply_ge65': '1939',
 'total_drug_cost': '1305.29',
 'total_drug_cost_ge65': '1305.29'}

I'm interested to know distribution of average drug cost for any one drug. The total drug cost is `total_drug_cost`. It's less clear what column to use for the total distribution, but `total_day_supply` seems most correct.

In [146]:
# Create a new column 'average_drug_day_cost'
import numpy as np
import pandas as pd

df = pd.read_json(data_string)
df.average_drug_day_cost = df['total_day_supply']/df['total_drug_cost']

# What are the most popular drugs perscribed? Using generic name here but drug name may also be interesting.
grouped = df.groupby('generic_name')
top_20_drug_names = grouped.size().sort_values(ascending=False)[0:20].keys()
'WARFARIN SODIUM' in top_20_drug_names

True

In [151]:
grouped

In [158]:
groups = grouped.groups

In [159]:
top_20_groups = []
for group_name in groups:
    if group_name in top_20_drug_names:
        top_20_groups.append(groups[group_name])


In [160]:
# import numpy as np
# import matplotlib.pyplot as plt
# %matplotlib inline

# x = first_group.average_drug_day_cost
# y = first_group.total_claim_count

# plt.scatter(x, y, alpha=0.5)
# plt.show()

In [161]:
# print(first_group.average_drug_day_cost.min())
# first_group.average_drug_day_cost.max()

0.557529379612


5.7188244638602059

In [196]:
# for every item in every group, I want to save the group's name, index and the avg drug cost value
results = []

for idx, group in enumerate(top_20_groups):
    group_rows = df.iloc[group]
    group_rows.average_drug_day_cost = group_rows['total_day_supply']/group_rows['total_drug_cost']    
    drug_name = group_rows.iloc[[0]].generic_name.values[0]
    print(drug_name)
    for value in group_rows.average_drug_day_cost:
        results.append([idx, drug_name, value])
        
len(results)

WARFARIN SODIUM
OMEPRAZOLE
ATORVASTATIN CALCIUM
BUPROPION HCL
HYDROCHLOROTHIAZIDE
ALBUTEROL SULFATE
SIMVASTATIN
NIFEDIPINE
AMLODIPINE BESYLATE
LEVOTHYROXINE SODIUM
LOSARTAN POTASSIUM
VALSARTAN
GABAPENTIN
HYDROCODONE/ACETAMINOPHEN
METFORMIN HCL
CLOPIDOGREL BISULFATE
TRAMADOL HCL
PANTOPRAZOLE SODIUM
METOPROLOL TARTRATE
GLIPIZIDE


244

In [197]:
import csv

my_df = pd.DataFrame(results)
my_df.to_csv('drugs/average_drug_costs.csv', index=False, header=True)